In [2]:
import sys,os,os.path

sys.path.append("../../")   # cite IC from parent directory
                            # NOTE if you can't import IC stuff, its because of the
                            # above line
#sys.path.append(os.path.expanduser('~/code/eol_hsrl_python'))
os.environ['ICTDIR']='/home/e78368jw/Documents/NEXT_CODE/IC'

import matplotlib.pyplot as plt
import pandas as pd
import numpy  as np
import tables as tb
import IC.invisible_cities.io.dst_io                           as     dstio
import IC.invisible_cities.io.mcinfo_io as mcio
from    IC.invisible_cities.core.core_functions   import shift_to_bin_centers
import iminuit,probfit

import scipy.special as special
from scipy.stats import skewnorm
from scipy.optimize import curve_fit

from scipy.integrate import quad

from Tl_studies.functions import isaura_analyse as isa

### This notepad only exists because I forgot to collect efficiency errors earlier

In [3]:
print("Efficiency error port 1a:")

er_1a = isa.ratio_error(1.06, 1057, 100009)
print(er_1a)
er_1b = isa.ratio_error(1.01, 968, 95548)
print(er_1b)
er_2a = isa.ratio_error(0.99, 706, 71041)
print(er_2a)
er_2b = isa.ratio_error(0.97, 925, 95437)
print(er_2b)

Efficiency error port 1a:
0.03277565412795216
0.032626655104977566
0.037443823049482225
0.032047590376821734


In [4]:
# lets speed this up a little
port1a_events = [67697, 39773, 23119, 2485, 1057]
cut_type = ['Fiducial', 'One track', 'overlap', 'ROI', 'eblob 2']

def efficiency_errors(events, tot_events, cut_type):

    for i in range(len(events)):
        if (i==0):
            print(cut_type[i])
            # calculate efficiency first
            effic = (events[i]/tot_events)*100
            
            error = isa.ratio_error(effic, events[i], tot_events)
            print(effic, error)
        else:
            print(cut_type[i])

            # calculate efficiency (relative)
            effic = (events[i]/events[i-1])*100

            error = isa.ratio_error(effic, events[i], events[i-1])
            print(effic, error)


In [5]:
efficiency_errors(port1a_events, 100009, cut_type)

Fiducial
67.69090781829635 0.3368995142702092
One track
58.75149563496167 0.37117922836570405
overlap
58.12737284087195 0.4807283118246525
ROI
10.748734806868809 0.22691517392233537
eblob 2
42.53521126760563 1.5619681806362395


In [6]:
port1b_events = [63709, 37954, 22802, 2342, 968]

efficiency_errors(port1b_events, 95548, cut_type)

Fiducial
66.67748147527945 0.34104937504624
One track
59.57400053367656 0.3862860768041564
overlap
60.07798914475417 0.5033785755877664
ROI
10.271028857117797 0.22286979693553297
eblob 2
41.33219470538002 1.5793243793393685


In [7]:
port2a_events = [47162, 29073, 15630, 1638, 706]

efficiency_errors(port2a_events, 71041, cut_type)

Fiducial
66.38701594853677 0.39431814071290505
One track
61.644968406768164 0.4596569230270024
overlap
53.76122175214116 0.5332281289034668
ROI
10.479846449136277 0.2721695289629459
eblob 2
43.101343101343105 1.9404850807885976


In [8]:
port2b_events = [64586, 38157, 20969, 2118, 925]

efficiency_errors(port2b_events, 95437, cut_type)

Fiducial
67.67396292842399 0.3448142690699429
One track
59.07936704548974 0.38146583707455967
overlap
54.95452996828891 0.4724074917311212
ROI
10.100624731746864 0.23029268846191944
eblob 2
43.67327667610954 1.721206228119172


In [8]:
# doing the full error system here instead
def binom_efficiency_errors(events, tot_events, cut_type):
    for i in range(len(events)):
        if (i==0):
            print(cut_type[i])
            epsilon = events[i] / tot_events
            # number of events within standard deviation of the number of passing events
            error_efficiency = np.sqrt((epsilon*(1-epsilon))*tot_events)

            # getting error on efficiency from this:
            error = 100*(error_efficiency/tot_events)
            print(epsilon * 100, error)
        else:
            print(cut_type[i])
            # relative
            epsilon = events[i] / events[i-1]
            # number of events within standard deviation of the number of passing events
            error_efficiency = np.sqrt((epsilon*(1-epsilon))*events[i-1])

            # getting error on efficiency from this:
            error = 100*(error_efficiency/events[i-1])
            print(epsilon * 100, error)

In [26]:
binom_efficiency_errors(port1a_events, 100009, cut_type)

Satellite Cut
54.859062684358406 0.15735840158156605
Fiducial
71.74285505978419 0.19222493982490932
One track
49.851375727242704 0.25202000968779276
overlap
56.10539190704311 0.3542715295759704
ROI
11.063675174856936 0.29896156745119534


In [28]:
binom_efficiency_errors(port1b_events, 95548, cut_type)

Satellite Cut
59.87775777619626 0.15856763974972476
Fiducial
70.280710340488 0.1910705345493964
One track
52.060483971250214 0.2491376060799884
overlap
57.53117087851718 0.34164180954707696
ROI
10.24661629162169 0.27634289865371964


In [29]:
binom_efficiency_errors(port2a_events, 71041, cut_type)

Satellite Cut
46.067763685758926 0.18701147108997798
Fiducial
72.93977449812081 0.24558102948729998
One track
50.261823970508146 0.3236150744988559
overlap
56.12602100350058 0.4530344309078363
ROI
10.216810216810217 0.3690783942525689


In [30]:
binom_efficiency_errors(port2b_events, 95437, cut_type)

Satellite Cut
48.085124218070554 0.16173085824514208
Fiducial
72.11871608812186 0.20932283928738124
One track
48.59499637418419 0.2747329314291579
overlap
54.86538581110489 0.39239237089392043
ROI
9.723481414324569 0.31540280097135814
